In [ ]:
### SETUP load the overlay
from pynq import Overlay

overlay = Overlay("/home/xilinx/jupyter_notebooks/DEPLOYMENT_DEMO/deployment_linear_PYNQ_Z2.bit")

In [ ]:
# ref: https://pynq.readthedocs.io/en/v2.5/overlay_design_methodology/overlay_tutorial.html
# ref: http://www.fpgadeveloper.com/2018/03/how-to-accelerate-a-python-function-with-pynq.html

print("loading...")

from pynq import DefaultIP
import numpy as np
import time

# the PARSE_FILES class is instantiated once and the all files required for computing the geometric values and test predictions
# may be loaded, stored and (saved - if required)
class parse_files():
    def __init__(self):
        #super().__init__()
        self.no_variables = None
        self.no_variables_int = None
        self.no_test_vectors = None
        self.no_test_vectors_int= None
        self.no_classes_int = None
        
        # other variables and arrays containing details on the training model and testing set
        self.n_svs_data_int = None
        self.testing_mat_fi_data_uint16 = None
        self.testing_labels_data_int = None
        
        # these are for each classifier and will need updated several times (for each training model)
        self.svs_fi_data_uint16 = None
        self.coeff_fi_data_uint32 = None
        self.offset_fi_data_uint32 = None
        
        
    def get_ds_details(self):
        # read in the dataset details
        f = open("ds_details.dat","r")

        contents = f.read()
        ds_details_data = contents.split()
        x = np.array(ds_details_data)
        ds_details_data_uint32 = np.asarray(x,np.uint32)
        #print(type(ds_details_data_uint32[0]))

        # no_variables
        self.no_variables = ds_details_data_uint32[0]
        self.no_variables_int = self.no_variables
        # (single-precision floating point 32 bit representation as an integer)
        
        # no_test_vectors
        self.no_test_vectors = ds_details_data_uint32[1]
        self.no_test_vectors_int = self.no_test_vectors

        # number of classes
        self.no_classes_int = ds_details_data_uint32[2]
    
    def get_no_svs(self):
        # read file containing the number of support vectors for each classifier
        f = open("n_svs.dat","r")

        contents = f.read()
        n_svs_data = contents.split()
        x = np.array(n_svs_data)
        self.n_svs_data_int = np.asarray(x,np.uint32)

        f.close()
        
    def get_testing_matrix(self):
        f = open("test_matrix_fi.dat","r")

        contents = f.read()
        testing_mat_fi_data = contents.split()
        x = np.array(testing_mat_fi_data)
        #self.testing_mat_data_uint16 = x.astype(uint16)
        self.testing_mat_fi_data_uint16 = np.asarray(x, np.uint16)
        
        f.close()
        
    #def get_kernel_parameters(self):
        
    def get_testing_labels(self):
        # for self checking Python tests
        f = open("test_labels.dat","r")

        contents = f.read()
        testing_labels_data = contents.split()
        x = np.array(testing_labels_data)
        #testing_labels_data_float = np.asfarray(x,np.float32)
        #self.testing_labels_data_int = testing_labels_data_float.astype(int)
        self.testing_labels_data_int = np.asarray(x, np.uint8)

        f.close()
        
        f = open("test_predictions_libsvm.dat","r")
        
        contents = f.read()
        testing_labels_data = contents.split()
        x = np.array(testing_labels_data)
        #testing_labels_data_float = np.asfarray(x,np.float32)
        #self.testing_labels_data_int = testing_labels_data_float.astype(int)
        self.test_predictions_libsvm = np.asarray(x, np.uint8)
        
        f.close()
        
    def get_support_vectors(self, current_classifier):
        # return support vectors for a particular classifier
        file_ext = ".dat"
        svs_file_name = "svs_fi_"
        svs_file_name_new = svs_file_name + str(current_classifier) + file_ext
        
        f = open(svs_file_name_new,"r")

        contents = f.read()
        svs_fi_data = contents.split()
        x = np.array(svs_fi_data)
        self.svs_fi_data_uint16 = np.asarray(x,np.uint16)
    
        f.close()
        
    def get_sv_coeffs(self, current_classifier):
        # store the support vector coefficients for a classifier
        file_ext = ".dat"
        coeffs_file_name = "coeffs_fi_"
        coeffs_file_name_new = coeffs_file_name + str(current_classifier) + file_ext

        f = open(coeffs_file_name_new,"r")

        contents = f.read()
        coeffs_fi_data = contents.split()
        x = np.array(coeffs_fi_data)
        self.coeffs_fi_data_uint32 = np.asarray(x,np.uint32)
    
        f.close()
        
    def get_offset(self, current_classifier):
        # store the offset for a classifier
        file_ext = ".dat"        
        offset_file_name  = "offset_fi_"
        offset_file_name_new = offset_file_name + str(current_classifier) + file_ext
        
        f = open(offset_file_name_new,"r")

        offset_fi_data = f.read()
        self.offset_fi_data_uint32 = np.asarray(offset_fi_data,np.uint32)
    
        f.close()
    
# the GEOMETRIC_VALUES_DRIVER class is instantiated once for each "geometric_values" IP core
# member functions include loading data to IP core AXI-lite slave interfaces for general design
# parameters and generating the contigous buffers to transfer through DMA to the AXI stream (AXIS) 
# ports on the IP
# INHERITS FROM PARSE_FILES
from pynq import MMIO

import pynq.lib.dma

from pynq import allocate
#from pynq import Xlnk
#xlnk = Xlnk()

class deployment_driver(parse_files):
    def __init__(self):
        #super().__init__()
             
        # current classifier we are calculating the geometric values for
        self.current_classifier = None
        
        self.geometric_values_out = None
        
        # get parameters from dat files which are general to all classifiers - i.e. not the support vectors, coefficient or offset
        self.get_ds_details()
        self.get_no_svs()
        self.get_testing_matrix()
        
        # used for parallel processing of geometric values
        self.classifier_indices = []
        self.no_classifiers = None
        
        # LISTS
        self.dma_data_instances = []#contains the support vectors followed immediately by testing matrix in C standard type contigous memory
        self.dma_cf_instances = []
        self.dma_ds_instances = []
        self.dma_gv_instances = []
        
        geometric_values_1 = overlay.geometric_values_1
        geometric_values_2 = overlay.geometric_values_2
        geometric_values_3 = overlay.geometric_values_3
        geometric_values_4 = overlay.geometric_values_4
        geometric_values_5 = overlay.geometric_values_5
        geometric_values_6 = overlay.geometric_values_6

        self.g_v_dispatcher = {
            1: geometric_values_1,
            2: geometric_values_2,
            3: geometric_values_3,
            4: geometric_values_4,
            5: geometric_values_5,
            6: geometric_values_6,
        }
        
        self.geometric_values_all = None
        self.test_predictions = None
        
        self.geometric_values_time = 0
        self.test_predictions_time = 0
        
        ## TEST ##
        self.tm_buffer = None
        
        
    def dma_init(self, no_classifiers):
        # initialise buffers not required to change on each iteration
        #self.test_matrix_buffer = allocate(shape=(self.no_test_vectors_int,self.no_variables_int), dtype=np.uint32)
        #np.copyto(self.test_matrix_buffer,self.testing_mat_data_float_IEEE754)
        
        # store all geometric values here
        self.geometric_values_all = np.zeros(shape=(self.no_test_vectors_int,int(no_classifiers)), dtype=np.uint32)
        self.test_predictions = np.zeros(shape=(self.no_test_vectors_int,), dtype=np.uint8)
        
        self.tm_buffer = allocate(shape=(self.no_test_vectors_int*self.no_variables_int,), dtype=np.uint16)
        np.copyto(self.tm_buffer,self.testing_mat_fi_data_uint16)
        
    def dma_delete(self):
        self.tm_buffer.close()    
        
    def dma_transfer_parallel(self, no_classifiers):
        # instantiate all DMAs - parallel design
        for n1 in range(6):
            self.dma_data_instances.append(self.g_v_dispatcher[n1+1].dma_data)
            self.dma_cf_instances.append(self.g_v_dispatcher[n1+1].dma_cf)
            self.dma_ds_instances.append(self.g_v_dispatcher[n1+1].dma_ds)
            self.dma_gv_instances.append(self.g_v_dispatcher[n1+1].dma_gv)
            
        geometric_values_buffer_1 = allocate(shape=(self.no_test_vectors_int,1), dtype=np.uint32)
        geometric_values_buffer_2 = allocate(shape=(self.no_test_vectors_int,1), dtype=np.uint32)
        geometric_values_buffer_3 = allocate(shape=(self.no_test_vectors_int,1), dtype=np.uint32)
        geometric_values_buffer_4 = allocate(shape=(self.no_test_vectors_int,1), dtype=np.uint32)    
        geometric_values_buffer_5 = allocate(shape=(self.no_test_vectors_int,1), dtype=np.uint32)    
        geometric_values_buffer_6 = allocate(shape=(self.no_test_vectors_int,1), dtype=np.uint32)    
    
        geo_values_dispatcher = {
            1: geometric_values_buffer_1,
            2: geometric_values_buffer_2,
            3: geometric_values_buffer_3,
            4: geometric_values_buffer_4,
            5: geometric_values_buffer_5,
            6: geometric_values_buffer_6,
        }
        
        # accumulate with time taken to transfer data to DMA in each classifier
        dma_transfer_time = 0
                
        # iterate over only required classifiers
        for n1 in range(6):
            if(n1 < len(self.classifier_indices)):
                #print("test")
                current_classifier = self.classifier_indices[n1]
                print("current_classifier: ", current_classifier)

                # get training model for current classifier to compute geometric values for this classifier
                # support vectors:
                self.get_support_vectors(current_classifier)
                # offset:
                self.get_offset(self.classifier_indices[n1])
                # support vector coefficients:
                self.get_sv_coeffs(self.classifier_indices[n1])

                # no_svs is obtained at start from one file
                # length of support vectors plus length of testing matrix by 256 variables
                svs_length = self.n_svs_data_int[self.classifier_indices[n1]-1] * self.no_variables_int
                testing_matrix_length = self.no_test_vectors_int * self.no_variables_int
                data_stream_length = svs_length + testing_matrix_length
                # length of coeffs plus one (for the offset)
                coeffs_stream_length = self.n_svs_data_int[self.classifier_indices[n1]-1] + 1
        
                #self.data_buffer = allocate(shape=(data_stream_length,), dtype=np.uint16)
            
                svs_buffer = allocate(shape=(svs_length,), dtype=np.uint16)
                coeffs_buffer = allocate(shape=(coeffs_stream_length,), dtype=np.uint32)
                ds_buffer = allocate(shape=(3,), dtype=np.uint32)
    
                #np.copyto(self.data_buffer[0:svs_length],self.svs_fi_data_uint16)
                #np.copyto(self.data_buffer[svs_length:data_stream_length],self.testing_mat_fi_data_uint16)
            
                np.copyto(svs_buffer,self.svs_fi_data_uint16)
                np.copyto(coeffs_buffer[0:coeffs_stream_length-1], self.coeffs_fi_data_uint32)
                coeffs_buffer[coeffs_stream_length-1] = self.offset_fi_data_uint32

                ds_buffer[0] = self.n_svs_data_int[self.classifier_indices[n1]-1]
                ds_buffer[1] = self.no_variables_int
                ds_buffer[2] = self.no_test_vectors_int
                
                
                # TEMP
                #offset = 0
                #print("sv 1")
                #print("Control: " + hex(self.dma_data_instances[n1].read(0x0 + offset)))
                #print("Status : " + hex(self.dma_data_instances[n1].read(0x4 + offset)))                
                #offset = 0
                #print("CF 1")
                #print("Control: " + hex(self.dma_cf_instances[n1].read(0x0 + offset)))
                #print("Status : " + hex(self.dma_cf_instances[n1].read(0x4 + offset)))                
                # TEMP                
                
                
                # transfer to DMA
                start_time = time.time()
                #self.dma_data_instances[n1].sendchannel.transfer(self.data_buffer)
                self.dma_cf_instances[n1].sendchannel.transfer(coeffs_buffer)
                self.dma_ds_instances[n1].sendchannel.transfer(ds_buffer)
                self.dma_gv_instances[n1].recvchannel.transfer(geo_values_dispatcher[n1+1])
                
                self.dma_data_instances[n1].sendchannel.transfer(svs_buffer)
                self.dma_data_instances[n1].sendchannel.wait()
                self.dma_data_instances[n1].sendchannel.transfer(self.tm_buffer)
                
                dma_transfer_time = dma_transfer_time + time.time() - start_time
                #print(dma_transfer_time)
                
                
                # TEMP
                #offset = 0
                #print("sv 2")
                #print("Control: " + hex(self.dma_data_instances[n1].read(0x0 + offset)))
                #print("Status : " + hex(self.dma_data_instances[n1].read(0x4 + offset)))                
                #offset = 0
                #print("CF 2")
                #print("Control: " + hex(self.dma_cf_instances[n1].read(0x0 + offset)))
                #print("Status : " + hex(self.dma_cf_instances[n1].read(0x4 + offset)))                  
                # TEMP

                #self.dma_ds_instances[n1].sendchannel.transfer(ds_buffer)
                #self.dma_gv_instances[n1].recvchannel.transfer(geo_values_dispatcher[n1+1])
                
                #del data_buffer
                coeffs_buffer.close()
                ds_buffer.close()
            else:
                break
                
        start_time = time.time()
                
        for n1 in range(6):
            if(n1 < len(self.classifier_indices)):
                self.dma_data_instances[n1].sendchannel.wait()
                self.dma_cf_instances[n1].sendchannel.wait()
                self.dma_ds_instances[n1].sendchannel.wait()
                self.dma_gv_instances[n1].recvchannel.wait()
            else:
                break
        
        elapsed_time = time.time() - start_time + dma_transfer_time
        self.geometric_values_time = self.geometric_values_time + elapsed_time
        #print("GEOMETRIC VALUES TIME: ")
        #print(elapsed_time)
                
        for n1 in range(6):
            if(n1 < len(self.classifier_indices)):        
                self.geometric_values_all[:,self.classifier_indices[n1]-1] = geo_values_dispatcher[n1+1][:,0]
                geo_values_dispatcher[n1+1].close()        
    
    def geometric_values_driver(self):
        # get training model for current classifier to compute geometric values for this classifier
        #self.get_support_vectors(current_classifier)
        #self.get_sv_coeffs(current_classifier)
        #self.get_offset(current_classifier)
        
        #self.dma_transfer(current_classifier)
        
        # generate require classifier indices in an 8-length array - there are currently 8 instances of geometric values
        # e.g. [1,2,3,4,5,6,7,8] then [9,10] if more than 8 classifiers or just [1,2,3,4,5,6]

        # get no_classifiers
        no_classifiers = self.no_classes_int * (self.no_classes_int - 1) / 2
        self.no_classifiers = no_classifiers
        
        self.dma_init(no_classifiers)

        current_classifier = 1
        done = 0
        
        while(done == 0):
            # generate indices - reset to length zero
            init_classifier = current_classifier
            # (init is the first classifier for the next batch of parallel processing)
            self.classifier_indices = []
            for n1 in range(6):
                if(current_classifier < (no_classifiers + 1)):
                    self.classifier_indices.append(init_classifier + n1)
                    #self.classifier_indices[n1] = current_classifier + n1
                    #if(n1 == 0):
                    #    self.classifier_indices[0] = current_classifier + n1
                    #else:
                    #    np.append(self.classifier_indices, current_classifier + n1)
                        
                    current_classifier = current_classifier + 1
            
            #print("current (next): ", current_classifier)
            if((current_classifier-1) == int(no_classifiers)):
                done = 1
            
            # call dma transfer - parallel calculate geometric values
            self.dma_transfer_parallel(no_classifiers)
        
        self.dma_delete()
        
    def test_predictions_driver(self):     
        no_classes = self.no_classes_int
        no_test_vectors = self

        dma_gv = overlay.test_predictions_1.dma_gv
        dma_ds = overlay.test_predictions_1.dma_ds
        dma_tp = overlay.test_predictions_1.dma_tp

        ge_values_buffer = allocate(shape=(self.no_test_vectors,int(self.no_classifiers)), dtype=np.uint32)
        dataset_buffer = allocate(shape=(2,1), dtype=np.uint32)

        np.copyto(ge_values_buffer,self.geometric_values_all)

        dataset_buffer[0] = self.no_classes_int
        dataset_buffer[1] = self.no_test_vectors

        test_predictions_out_buffer = allocate(shape=(self.no_test_vectors,1), dtype=np.uint8)

        start_time = time.time()
            
        # transfer to DMA
        dma_gv.sendchannel.transfer(ge_values_buffer)
        dma_ds.sendchannel.transfer(dataset_buffer)
        dma_tp.recvchannel.transfer(test_predictions_out_buffer)

        dma_gv.sendchannel.wait()
        dma_ds.sendchannel.wait()
        dma_tp.recvchannel.wait()
        
        elapsed_time = time.time() - start_time
        self.test_predictions_time = elapsed_time
        #print("TEST PREDICTIONS TIME: ")
        #print(elapsed_time)

        self.test_predictions = test_predictions_out_buffer

        # delete memory on heap to avoid memory leakage
        ge_values_buffer.close()
        dataset_buffer.close()
        test_predictions_out_buffer.close()
        
    def get_test_predictions(self):
        # get geometric values
        self.geometric_values_time = 0
        self.test_predictions_time = 0
        start_time = time.time()  
        
        self.geometric_values_driver()
        
        #elapsed_time = time.time() - start_time
        #print("TIME (geometric values total): ")
        #print(elapsed_time)
        
        # use geometric values to compute test predictions
        #start_time = time.time()  
        
        self.test_predictions_driver()
        
        elapsed_time = time.time() - start_time
        print("\nTIME TOTAL (WITH FILE READS): ", elapsed_time)
        
        print("TIME TO RECORD (NOT INCLUDING FILE READS): ", self.geometric_values_time + self.test_predictions_time)

print("\ndone")

In [ ]:
geometric_values_driver_inst = deployment_driver()

#start_time = time.time()

geometric_values_driver_inst.get_test_predictions()

#elapsed_time = time.time() - start_time
#print("TIME (TOTAL): ")
#print(elapsed_time)

In [ ]:
# check the accuracy of the prediction and simlarity to libsvm result
geometric_values_driver_inst.get_testing_labels()

# track errors to compute accuracy of precdiction
err_count = 0
# track differences to libsvm - this indicates issues with the numerical precision of the algorithm
disimilarity_count = 0

for i in range(geometric_values_driver_inst.no_test_vectors_int):
    if(geometric_values_driver_inst.test_predictions[i] != geometric_values_driver_inst.testing_labels_data_int[i]):
        err_count = err_count + 1
    if(geometric_values_driver_inst.test_predictions[i] != geometric_values_driver_inst.test_predictions_libsvm[i]):
        disimilarity_count = disimilarity_count + 1
        
print("accuracy = ", (geometric_values_driver_inst.no_test_vectors_int - err_count) / geometric_values_driver_inst.no_test_vectors_int * 100, "%")
print("similarity = ", (geometric_values_driver_inst.no_test_vectors_int - disimilarity_count) / geometric_values_driver_inst.no_test_vectors_int * 100, "%")

In [ ]:
geometric_values_driver_inst.test_predictions